In [1]:
import numpy as np
import pandas as pd
from pylsl import StreamInlet, resolve_byprop
from scipy.signal import welch
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

# Step 1: Connect to Muse LSL Stream
print("Looking for EEG streams...")
streams = resolve_byprop('type', 'EEG')
inlet = StreamInlet(streams[0])

Looking for EEG streams...


2025-03-04 19:03:20.508 (   0.274s) [          7916BB]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2025-03-04 19:03:20.508 (   0.274s) [          7916BB]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2025-03-04 19:03:20.508 (   0.274s) [          7916BB]      netinterfaces.cpp:102   INFO| 	IPv4 addr: 7f000001
2025-03-04 19:03:20.508 (   0.274s) [          7916BB]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2025-03-04 19:03:20.508 (   0.274s) [          7916BB]      netinterfaces.cpp:105   INFO| 	IPv6 addr: ::1
2025-03-04 19:03:20.508 (   0.274s) [          7916BB]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2025-03-04 19:03:20.508 (   0.274s) [          7916BB]      netinterfaces.cpp:105   INFO| 	IPv6 addr: fe80::1%lo0
2025-03-04 19:03:20.508 (   0.274s) [          7916BB]      netinterfaces.cpp:91    I

In [6]:
import numpy as np
import pickle

from scipy.signal import welch
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler
print("Looking for EEG streams...")
streams = resolve_byprop('type', 'EEG')
inlet = StreamInlet(streams[0])

def collect_data(duration=10, fs=256, samples=3):  
    data = []
    for i in range(samples):
        print(f"Collecting sample {i+1}/{samples}...")
        trial = []
        for _ in range(duration * fs):
            sample, _ = inlet.pull_sample()
            trial.append(sample)
        data.append(np.array(trial))
    return np.array(data)

def extract_features(eeg_data, fs=256):
    all_features = []
    for trial in eeg_data:
        features = []
        for channel in trial.T:
            freqs, psd = welch(channel, fs=fs, nperseg=fs)
            features.append(psd[:30])  # First 30 frequency bins
        all_features.append(np.concatenate(features))
    return np.array(all_features)

# Train on multiple samples
eeg = collect_data(samples=3)
features = extract_features(eeg)

scaler = StandardScaler()
X = scaler.fit_transform(features)

clf = OneClassSVM(nu=0.05, kernel="rbf")  # Reduced sensitivity
clf.fit(X)

# Save model
with open("mind_id_model.pkl", "wb") as f:
    pickle.dump((clf, scaler), f)

print("Model trained with multiple samples. Try verifying again.")


Looking for EEG streams...
Model trained with multiple samples. Try verifying again.


In [2]:
import numpy as np
import pickle
from pylsl import StreamInlet, resolve_byprop
from scipy.signal import welch

# Load trained model & scaler
with open("mind_id_model.pkl", "rb") as f:
    clf, scaler = pickle.load(f)

# Step 1: Connect to Muse EEG Stream
print("Looking for EEG streams...")
streams = resolve_byprop('type', 'EEG')
inlet = StreamInlet(streams[0])

# Step 2: Collect EEG Data
def collect_data(duration=10, fs=256):
    samples = []
    for _ in range(duration * fs):
        sample, _ = inlet.pull_sample()
        samples.append(sample)
    return np.array(samples)

# Step 3: Extract Features
def extract_features(eeg_data, fs=256):
    features = []
    for channel in eeg_data.T:
        freqs, psd = welch(channel, fs=fs, nperseg=fs)
        features.append(psd[:30])  # First 30 frequency bins
    return np.concatenate(features)

# Step 4: Verify User with Confidence Score
def verify_user():
    print("Verifying user...")
    eeg = collect_data()
    features = extract_features(eeg)
    features = scaler.transform([features])

    score = clf.decision_function(features)[0]  # Confidence score

    print(f"Confidence Score: {score:.4f}")  # Print the decision score

    if score > -0.5:  # Adjust threshold as needed
        print("✅ Access Granted: User verified.")
    else:
        print("❌ Access Denied: Unknown User.")

verify_user()


Looking for EEG streams...
Verifying user...
Confidence Score: -0.0556
✅ Access Granted: User verified.


2025-03-04 19:30:35.079 (1634.840s) [R_Muse          ]      data_receiver.cpp:344    ERR| Stream transmission broke off (Input stream error.); re-connecting...


In [ ]:
authenticate_user()